In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
def plot(column):
    if data[column].dtype == 'object':
        f, axes = plt.subplots(1,1,figsize=(15,5))
        sns.countplot(x=column, hue='status', data = data)
        plt.xticks(rotation=90)
        plt.suptitle(column,fontsize=20)
        plt.show()
    else:
        g = sns.FacetGrid(data, row="status", margin_titles=True, aspect=4, height=3)
        g.map(plt.hist,column)
        plt.suptitle(column)
        plt.show()
    plt.show()

In [ ]:
for column in data.columns:
    plot(column)

In [ ]:
X = data[['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s',
       'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p']]
y = data['status']

In [ ]:
data.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def datacleaner(x,y):
    y = y.map({'Not Placed':0, 'Placed':1})
    x = pd.get_dummies(x,columns =['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation'], dtype = int, drop_first=True)
    return x, y

In [ ]:
datacleaner(X_train, y_train)[0]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

for i in range(1,10):
    model = DecisionTreeClassifier(criterion = 'entropy', max_depth = i)
    model.fit(datacleaner(X_train, y_train)[0],datacleaner(X_train, y_train)[1])


    y_pred = model.predict(datacleaner(X_test, y_test)[0])
    from sklearn import metrics

    cm = metrics.confusion_matrix(datacleaner(X_test, y_test)[1], y_pred)
    print('Confusion Matrix - score with max depth of '+ str(i) + ': '+str(metrics.accuracy_score(datacleaner(X_test, y_test)[1],y_pred)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


print('Decision tree with max_depth of '+str(3))
model = DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)
model.fit(datacleaner(X_train, y_train)[0],datacleaner(X_train, y_train)[1])


y_pred = model.predict(datacleaner(X_test, y_test)[0])
from sklearn import metrics

cm = metrics.confusion_matrix(datacleaner(X_test, y_test)[1], y_pred)
plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Confusion Matrix - score with max depth of '+ str(i) + ': '+str(metrics.accuracy_score(datacleaner(X_test, y_test)[1],y_pred))
plt.title(all_sample_title, size = 15);
plt.show()
print(metrics.classification_report(datacleaner(X_test, y_test)[1],y_pred))
print('------------------------------')

In [ ]:
from sklearn import tree
from subprocess import check_call
from IPython.display import Image as PImage
from PIL import Image, ImageDraw, ImageFont

feature_names = datacleaner(X_train, y_train)[0].columns

with open("tree1.dot", 'w') as f:
     f = tree.export_graphviz(model,out_file=f,max_depth = 3,impurity = True ,feature_names = feature_names)
        
#Convert .dot to .png to allow display in web notebook
check_call(['dot','-Tpng','tree1.dot','-o','tree1.png'])

# Annotating chart with PIL
img = Image.open("tree1.png")
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf', 26)
draw.text((10, 0),'Decision tree',(0,0,255),font=font) # ImageFont object with desired font
img.save('sample-out.png')
PImage("sample-out.png")


In [ ]:
features_considered_by_decision_tree = ['ssc_p', 'hsc_p', 'degree_p', 'mba_p']
for feature in features_considered_by_decision_tree:
    plot(feature)